In [1]:
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, xyz2lab
from skimage.io import imsave
import numpy as np
import os
import random
import tensorflow as tf
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from PIL import Image
import glob
from keras.callbacks import TensorBoard

Using TensorFlow backend.


In [11]:
data = glob.glob('./data_faces/Train/*.jpg')
# img = load_img('./data_faces/Train/0A9kTN.jpg')

data = data[:4500]
print('Len data: ', len(data))
images = []
for items in data:
    images.append(img_to_array(load_img(items)))
images = np.array(images, dtype=float)

Len data:  4500


In [12]:
print('Len images: ', len(images))
print('Image shape: ', images[0].shape)

Len images:  4500
Image shape:  (256, 256, 3)


## 1.0/255 : Indicates that we are using a 24-bit RGB color space.  0-255 for each color channel

In [13]:
X_train = 1.0/255*images

print(X_train.shape)

(4500, 256, 256, 3)


In [14]:
model = Sequential()
model.add(InputLayer(input_shape=(256, 256, 1)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
model.add(UpSampling2D((2, 2)))

In [15]:
model.summary()
model.compile(optimizer='rmsprop',loss='mse')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 256, 256, 1)       0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 256, 256, 64)      640       
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 128, 128, 64)      36928     
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 128, 128, 128)     73856     
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 64, 64, 128)       147584    
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 64, 64, 256)       295168    
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 32, 32, 256)       590080    
__________

## We are using a ImageDataGenerator because the dataset is too big

In [16]:
datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=20,
        horizontal_flip=True)

In [17]:
4500/50

90.0

#### - [:,:,:,0] : Select the grayscale for training images (X_train)
#### - [:,:,:,1:] : Select the green-red and blue-yellow for labels images (Y_train)

In [18]:
batch_size = 50
def image_a_b_gen(batch_size):
    for batch in datagen.flow(X_train, batch_size=batch_size):
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield (X_batch.reshape(X_batch.shape+(1,)), Y_batch)

In [19]:
tensorboard = TensorBoard(log_dir="log/")
model.fit_generator(image_a_b_gen(batch_size), callbacks=[tensorboard], 
                    epochs=21, steps_per_epoch=90)

# Save model# Save  
model_json = model.to_json()
with open("./Models/model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("./Models/model.h5")

Epoch 1/21
90/90 [==============================] - 208s - loss: 0.1098   
Epoch 2/21
90/90 [==============================] - 208s - loss: 0.0109   
Epoch 3/21
90/90 [==============================] - 206s - loss: 0.0104   
Epoch 4/21
90/90 [==============================] - 205s - loss: 0.0104   
Epoch 5/21
90/90 [==============================] - 204s - loss: 0.0104   
Epoch 6/21
90/90 [==============================] - 207s - loss: 0.0111   
Epoch 7/21
90/90 [==============================] - 203s - loss: 0.0102   
Epoch 8/21
90/90 [==============================] - 207s - loss: 0.0102   
Epoch 9/21
90/90 [==============================] - 204s - loss: 0.0101   
Epoch 10/21
90/90 [==============================] - 201s - loss: 0.0101   
Epoch 11/21
90/90 [==============================] - 201s - loss: 0.0100   
Epoch 12/21
90/90 [==============================] - 201s - loss: 0.0099   
Epoch 13/21
90/90 [==============================] - 196s - loss: 0.0100   
Epoch 14/21
90/90 [==

In [23]:
data = glob.glob('./data_faces/Test/*.jpg')
# img = load_img('./data_faces/Train/0A9kTN.jpg')


color_me = []
for items in data:
    color_me.append(img_to_array(load_img(items)))

# for filename in os.listdir('./data_faces/Test/'):
#         color_me.append(img_to_array(load_img('../Test/'+filename)))
color_me = np.array(color_me, dtype=float)
color_me = rgb2lab(1.0/255*color_me)[:,:,:,0]
color_me = color_me.reshape(color_me.shape+(1,))

In [24]:
output = model.predict(color_me)
# El resultado es entre -1 y 1, por eso lo multiplicamos por 128
output = output * 128

In [25]:
for i in range(len(output)):
        cur = np.zeros((256, 256, 3))
        # Creamos un black RGB canvas para llenar los pixeles que tienen las 3 capas 
        # de RGB a 0
        cur[:,:,0] = color_me[i][:,:,0]
        cur[:,:,1:] = output[i]
        imsave("./results/img_"+str(i)+".png", lab2rgb(cur))

C:\Users\aleja\Anaconda3\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
C:\Users\aleja\Anaconda3\lib\site-packages\skimage\io\_io.py:132: UserWarning: ./results/img_30.png is a low contrast image
  warn('%s is a low contrast image' % fname)
C:\Users\aleja\Anaconda3\lib\site-packages\skimage\io\_io.py:132: UserWarning: ./results/img_236.png is a low contrast image
  warn('%s is a low contrast image' % fname)


In [ ]:
# model = Sequential()
# model.add(InputLayer(input_shape=(None, None, 1)))
# model.add(Conv2D(8, (3, 3), activation='relu', padding='same', strides=2))
# model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
# model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
# model.add(Conv2D(16, (3, 3), activation='relu', padding='same', strides=2))
# model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
# model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=2))
# model.add(UpSampling2D((2, 2)))
# model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
# model.add(UpSampling2D((2, 2)))
# model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
# model.add(UpSampling2D((2, 2)))
# model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))

In [ ]:
# model.summary()
# model.compile(optimizer='rmsprop',loss='mse')

In [ ]:
# model.fit(x=X, y=Y, batch_size=1, epochs=3000)
# print(model.evaluate(X, Y, batch_size=1))

In [ ]:
# output = model.predict(X)
# output = output * 128
# canvas = np.zeros((256, 256, 3))
# canvas[:,:,0] = X[0][:,:,0]
# canvas[:,:,1:] = output[0]
# imsave("img_result.png", lab2rgb(canvas))
# imsave("img_gray_scale.png", rgb2gray(lab2rgb(canvas)))